# Hyperparam

In [1]:
prune_n = 2
prune_m = 4
# note that 3:4 means that 3 out of 4 is REMOVED

prune_method = "sparsegpt"
# "magnitude", "wanda", "sparsegpt"

model_name = "google/gemma-3-270m"
# "facebook/opt-125m", "facebook/opt-350m", "facebook/opt-1.3b"

# "meta-llama/Llama-3.2-1B"

# "Qwen/Qwen2.5-0.5B", Qwen/Qwen2.5-1.5B"
# "google/gemma-3-270m", "google/gemma-3-1b-pt"  # <---- needs additional code for loading

## Autofill

In [ ]:
import os

sparsity_ratio: float = float(prune_n) / float(prune_m)
sparsity_type = f"{prune_n}:{prune_m}"

repository = 'wanda_vision'
repo_path = os.getcwd()

save_name = model_name.replace("/", "__") + f"-{prune_n}of{prune_m}"
model_save_path = f"{repo_path}/pruned_models/{prune_method}/{save_name}"

# Codes

In [ ]:
# !pip install -q --force-reinstall --no-cache-dir transformers
# !pip install -q --upgrade accelerate deepspeed
try:
    import lm_eval
except ImportError:
    !pip install -q lm_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 113.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!mkdir -p pruned_models

In [ ]:
from huggingface_hub import login
login("")

# Main

In [ ]:
print(f"Pruning '{model_name}' using '{prune_method}' method with {sparsity_type} sparsity (ratio = {sparsity_ratio}).")
print(f"Model is saved at {model_save_path}")

no_eval = False

import subprocess
try:
    if 'opt' not in model_name:
        subprocess.run([
            'python', 'main.py',
            '--model', model_name,
            '--prune_method', prune_method,
            '--sparsity_ratio', str(sparsity_ratio),
            '--sparsity_type', sparsity_type,
            '--save_model', model_save_path
        ], check=True)
    else:
        subprocess.run([
            'python', 'main_opt.py',
            '--model', model_name,
            '--prune_method', prune_method,
            '--sparsity_ratio', str(sparsity_ratio),
            '--sparsity_type', sparsity_type,
            '--save_model', model_save_path
        ], check=True)

except subprocess.CalledProcessError:
    no_eval = True

In [ ]:
if not no_eval:
    !torchrun --nproc-per-node=2 --no-python lm_eval \
        --model hf \
        --model_args pretrained={model_save_path},max_position_embeddings=4096 \
        --tasks wikitext,hellaswag,race,piqa,winogrande,arc_easy,arc_challenge,sciq \
        --batch_size auto \
        --output_path results